## Mixture of Fine-Tuned Cross-Modal Experts

### Minjun Kim, Heidi Lau, David Liu, Sunny Park

# 1. Setup Environment

In [ ]:
# Check GPU configuration
!nvidia-smi

# Install required libraries
!pip install torch torchvision torchaudio transformers datasets scikit-learn py7zr

Wed Dec 11 11:18:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              61W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
import os
import random, json

from PIL import Image

import torch
import torch.nn as nn
from torch.nn.functional import cross_entropy
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
import torch.nn.functional as F

import torchvision.transforms as T
from torchvision.datasets import CIFAR10, OxfordIIITPet

from transformers import BertModel, ViTModel, BertTokenizer, DistilBertModel
from datasets import load_dataset

from sklearn.model_selection import train_test_split

# 2. Data Preparation



In [ ]:
# Ensure directories exist
os.makedirs("datasets", exist_ok=True)

In [ ]:
!curl -L -o flickr8k https://www.kaggle.com/api/v1/datasets/download/adityajn105/flickr8k

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1061M  100 1061M    0     0   153M      0  0:00:06  0:00:06 --:--:--  216M


In [ ]:
!unzip flickr8k -d datasets/flickr8k

Archive:  flickr8k
replace datasets/flickr8k/Images/1000268201_693b08cb0e.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
# Define Image Transformations
image_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Task 1: Image Classification (CIFAR-10)
cifar10_dataset = CIFAR10(root="datasets", train=True, transform=image_transform, download=True)

# Task 2: Sentiment Analysis (SST-2)
sst2_dataset = load_dataset("glue", "sst2", split="train")

# Task 3: Image Captioning (Custom Flickr8k Loader)
class CustomFlickr8k(Dataset):
    def __init__(self, root, ann_file, transform=None):
        """
        Custom dataset loader for Flickr8k with captions.txt formatted as 'image,caption'.
        :param root: Root directory containing the 'images/' folder.
        :param ann_file: Path to the captions.txt file.
        :param transform: Transformations to apply to images.
        """
        self.root = root
        self.ann_file = ann_file
        self.transform = transform
        self.image_captions = []

        # Parse the captions file
        with open(ann_file, "r") as f:
            lines = f.readlines()[1:]
            for line in lines:
                parts = line.strip().split(",", 1)  # Split into image and caption
                if len(parts) == 2:
                    image_id, caption = parts
                    self.image_captions.append((image_id, caption))
                else:
                    print(f"Skipping malformed line: {line.strip()}")

    def __len__(self):
        return len(self.image_captions)

    def __getitem__(self, idx):
        image_id, caption = self.image_captions[idx]
        image_path = os.path.join(self.root, "Images", image_id)

        # Load the image
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, caption

# Setup Flickr8k Dataset
flickr8k_dataset = CustomFlickr8k(
    root="datasets/flickr8k",
    ann_file="datasets/flickr8k/captions.txt",
    transform=image_transform
)

# Task 4: Text Summarization (Samsum)
samsum_dataset = load_dataset("samsum", split="train", trust_remote_code=True)

# Task 5: Image Segmentation (Oxford-IIIT Pet Dataset)
pet_dataset = OxfordIIITPet(root="datasets", split="trainval", transform=image_transform, download=True)



Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Unified Multi-Task Dataset
class MultiTaskDataset(Dataset):
    def __init__(self, datasets):
        """
        Unified dataset wrapper for multi-task learning.
        :param datasets: A dictionary mapping task names to datasets.
        """
        self.datasets = datasets
        self.task_keys = list(datasets.keys())

        # Precompute dataset offsets for deterministic sampling
        self.dataset_offsets = {}
        offset = 0
        for task, dataset in datasets.items():
            self.dataset_offsets[task] = (offset, offset + len(dataset))
            offset += len(dataset)
        self.total_length = offset

    def __len__(self):
        return self.total_length

    def __getitem__(self, idx):
        # Find the corresponding task and local index
        for task, (start, end) in self.dataset_offsets.items():
            if start <= idx < end:
                dataset_idx = idx - start
                sample = self.datasets[task][dataset_idx]

                # Standardize output format
                if task == "classification":  # CIFAR-10
                    input_data, label = sample
                elif task == "sentiment":  # SST-2
                    input_data = sample["sentence"]
                    label = sample["label"]
                elif task == "captioning":  # Flickr8k
                    input_data, label = sample  # Input: image, Label: caption
                elif task == "summarization":  # Samsum
                    input_data = sample["dialogue"]
                    label = sample["summary"]
                elif task == "segmentation":  # Oxford-IIIT Pet
                    input_data, label = sample  # Input: image, Label: breed label

                return {"task": task, "input": input_data, "label": label}

        raise IndexError(f"Index {idx} out of range for MultiTaskDataset")

# Combine Datasets
multi_task_dataset = MultiTaskDataset({
    "classification": cifar10_dataset,
    "sentiment": sst2_dataset,
    "captioning": flickr8k_dataset,
    "summarization": samsum_dataset,
    "segmentation": pet_dataset
})

In [ ]:
def multi_task_collate_fn(batch):
    """
    Custom collate function to handle multi-task data with mixed types.
    :param batch: List of samples returned by the dataset.
    :return: A dictionary of grouped data by task.
    """
    grouped_batch = {"task": [], "input": {}, "label": {}}

    for sample in batch:
        task = sample["task"]
        input_data = sample["input"]
        label = sample["label"]

        # Append task
        grouped_batch["task"].append(task)

        # Group inputs and labels by task
        if task not in grouped_batch["input"]:
            grouped_batch["input"][task] = []
            grouped_batch["label"][task] = []

        grouped_batch["input"][task].append(input_data)
        grouped_batch["label"][task].append(label)

    # Convert tensors to batched tensors for tasks with tensor data
    for task in grouped_batch["input"]:
        if isinstance(grouped_batch["input"][task][0], torch.Tensor):
            grouped_batch["input"][task] = torch.stack(grouped_batch["input"][task])
        # Leave text data as lists

    for task in grouped_batch["label"]:
        if isinstance(grouped_batch["label"][task][0], torch.Tensor):
            grouped_batch["label"][task] = torch.stack(grouped_batch["label"][task])
        # Leave text labels as lists

    return grouped_batch


In [ ]:
# DataLoader
multi_task_loader = DataLoader(
    multi_task_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=multi_task_collate_fn
)

In [ ]:
# Verify batches
for batch_idx, batch in enumerate(multi_task_loader):
    print(f"Batch {batch_idx + 1}:")
    print(f"Tasks: {batch['task']}")  # List of task names
    print(f"Input type: {type(batch['input'])}")  # Tensor for images, list for text
    print(f"Label type: {type(batch['label'])}")  # Tensor or list depending on task

    if isinstance(batch['input'], torch.Tensor):  # If input is image data
        print(f"Input shape: {batch['input'].shape}")  # Tensor shape
    if isinstance(batch['label'], torch.Tensor):  # If labels are tensors
        print(f"Label shape: {batch['label'].shape}")  # Tensor shape

    if batch_idx == 2:  # Limit to 3 batches for verification
        break

Batch 1:
Tasks: ['summarization', 'summarization', 'sentiment', 'classification', 'classification', 'sentiment', 'summarization', 'sentiment']
Input type: <class 'dict'>
Label type: <class 'dict'>
Batch 2:
Tasks: ['classification', 'captioning', 'sentiment', 'captioning', 'sentiment', 'sentiment', 'summarization', 'sentiment']
Input type: <class 'dict'>
Label type: <class 'dict'>
Batch 3:
Tasks: ['sentiment', 'summarization', 'sentiment', 'sentiment', 'sentiment', 'classification', 'sentiment', 'classification']
Input type: <class 'dict'>
Label type: <class 'dict'>


Verify Datasets

In [ ]:
# # Verify CIFAR-10 Dataset
print("Verifying CIFAR-10 Dataset (Image Classification)...")
for idx in range(3):  # Check the first 3 samples
    image, label = cifar10_dataset[idx]
    print(f"Sample {idx + 1}:")
    print(f"Image shape: {image.shape}")  # Should be (3, 224, 224)
    print(f"Label: {label}")  # Integer label (0-9)
print("\n")

# Verify SST-2 Dataset
print("Verifying SST-2 Dataset (Sentiment Analysis)...")
for idx in range(3):  # Check the first 3 samples
    sample = sst2_dataset[idx]
    print(f"Sample {idx + 1}:")
    print(f"Text: {sample['sentence']}")  # Input text
    print(f"Label: {sample['label']}")  # 0 (negative) or 1 (positive)
print("\n")

# Verify Flickr8k Dataset
print("Verifying Flickr8k Dataset (Image Captioning)...")
flickr8k_dataset = CustomFlickr8k(
    root="datasets/flickr8k",
    ann_file="datasets/flickr8k/captions.txt",
    transform=image_transform
)
for idx in range(3):  # Check the first 3 samples
    try:
        image, caption = flickr8k_dataset[idx]
        print(f"Sample {idx + 1}:")
        print(f"Image shape: {image.shape}")  # Should be (3, 224, 224)
        print(f"Caption: {caption}")  # Caption as a string
    except Exception as e:
        print(f"Error with sample {idx + 1}: {e}")
print("\n")

# Verify samsum Dataset
print("Verifying samsum Dataset (Text Summarization)...")
for idx in range(3):  # Check the first 3 samples
    sample = samsum_dataset[idx]
    print(f"Sample {idx + 1}:")
    print(f"Dialogue: {sample['dialogue']}")  # Input article
    print(f"Summary: {sample['summary']}")  # Target summary
print("\n")

# Verify Oxford-IIIT Pet Dataset
print("Verifying Oxford-IIIT Pet Dataset (Image Classification)...")
for idx in range(10):  # Check the first 3 samples
    image, label = pet_dataset[idx]
    print(f"Sample {idx + 1}:")
    print(f"Image shape: {image.shape}")  # Should be (3, 224, 224)
    print(f"Label: {label}")  # Integer label corresponding to the pet breed
print("\n")



Verifying CIFAR-10 Dataset (Image Classification)...
Sample 1:
Image shape: torch.Size([3, 224, 224])
Label: 6
Sample 2:
Image shape: torch.Size([3, 224, 224])
Label: 9
Sample 3:
Image shape: torch.Size([3, 224, 224])
Label: 9


Verifying SST-2 Dataset (Sentiment Analysis)...
Sample 1:
Text: hide new secretions from the parental units 
Label: 0
Sample 2:
Text: contains no wit , only labored gags 
Label: 0
Sample 3:
Text: that loves its characters and communicates something rather beautiful about human nature 
Label: 1


Verifying Flickr8k Dataset (Image Captioning)...
Sample 1:
Image shape: torch.Size([3, 224, 224])
Caption: A child in a pink dress is climbing up a set of stairs in an entry way .
Sample 2:
Image shape: torch.Size([3, 224, 224])
Caption: A girl going into a wooden building .
Sample 3:
Image shape: torch.Size([3, 224, 224])
Caption: A little girl climbing into a wooden playhouse .


Verifying samsum Dataset (Text Summarization)...
Sample 1:
Dialogue: Amanda: I baked  coo

# 3. Define the Mixture-of-Modalities (MoM) Backbone

The MoM backbone includes modality-specific processing for text and image inputs.

In [ ]:
class MixtureOfModalities(nn.Module):
    def __init__(self):
        super(MixtureOfModalities, self).__init__()

    def forward(self, inputs):
        """
        Process inputs for the Mixture of Experts.
        :param inputs: Dictionary containing raw text or image input.
        :return: Formatted dictionary with 'prompt' and 'input' keys.
        """
        formatted_inputs = {}

        # Handle text input
        if "text" in inputs:
            text = inputs["text"]
            if "." in text:
                split_idx = text.index(".") + 1  # Split at the first period
                formatted_inputs["prompt"] = text[:split_idx].strip()
                formatted_inputs["input"] = text[split_idx:].strip() or None
            else:
                formatted_inputs["prompt"] = text.strip()
                formatted_inputs["input"] = None

        # Handle image input
        if "image" in inputs:
            image_input = inputs["image"]
            if "input" in formatted_inputs and formatted_inputs["input"]:
                formatted_inputs["input"] = {
                    "text": formatted_inputs["input"],
                    "image": image_input
                }
            else:
                formatted_inputs["input"] = image_input

        # Ensure at least the prompt is available
        if "prompt" not in formatted_inputs:
            raise ValueError("At least a 'prompt' is required for processing.")

        return formatted_inputs


# 4. Define the Task Classifier



### 4.1 Model Definition

The Task Classifier predicts a singular task. It uses a fully connected network with sigmoid activation for task probabilities.

In [ ]:
class TaskClassifier(nn.Module):
    def __init__(self, num_tasks=5):
        super(TaskClassifier, self).__init__()
        # Pretrained BERT model as the text encoder
        self.text_encoder = BertModel.from_pretrained("bert-base-uncased")
        # Fully connected classification head
        self.classifier = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Linear(512, num_tasks)
        )

    def forward(self, text_input):
        """
        Forward pass for the Task Classifier.
        :param text_input: Tokenized text input (BERT format).
        :return: Task logits and probabilities.
        """
        # Get BERT's pooled output
        text_features = self.text_encoder(**text_input).pooler_output  # Shape: (batch_size, 768)
        # Pass through the classification head
        task_logits = self.classifier(text_features)  # Shape: (batch_size, num_tasks)
        probabilities = F.softmax(task_logits, dim=-1)  # Convert to probabilities
        return task_logits, probabilities


### 4.2 Dataset Preparation

We’ll create a custom dataset with 125 prompts for each task.

In [ ]:
TASK_PROMPTS = {
    "captioning": [
        "Write a caption for this image.",
        "Describe what is happening in the picture.",
        "What is shown in the image?",
        "Explain the content of this image in a sentence.",
        "Provide a one-sentence description of this image.",
        "Describe the scene in the image.",
        "What does the image depict?",
        "Write a descriptive caption for the picture.",
        "Summarize the content of the image.",
        "Generate a caption for the given image.",
        "What activity is occurring in the picture?",
        "Provide a sentence describing the objects and scene in the image.",
        "What is the main focus of this image?",
        "Generate a natural language description of this image.",
        "Explain the image in your own words.",
        "What event is taking place in the picture?",
        "Describe the objects and actions in the image.",
        "Provide a short description of this photograph.",
        "Write a caption that explains the image.",
        "Summarize the activity or objects in the picture.",
        "What does the photograph show?",
        "Provide a single-sentence summary of this image.",
        "Describe this image in detail.",
        "Write a sentence about what is happening in the image.",
        "Summarize the visual content of the picture."
    ],
    "summarization": [
        "Summarize the main points of this document.",
        "Provide a brief summary of the text.",
        "What is the gist of this article?",
        "Condense the content of the document into a few sentences.",
        "Summarize the document in one or two lines.",
        "What are the key points discussed in this text?",
        "Provide a concise summary of the article.",
        "What is the main idea of the document?",
        "Generate a short summary of this article.",
        "Summarize the content in simple terms.",
        "Write a brief overview of this document.",
        "Extract the essential information from the text.",
        "What is the central idea of the article?",
        "Provide a short description of the document’s content.",
        "Summarize this article in plain language.",
        "Generate a few sentences that capture the main idea of this text.",
        "What is this document about?",
        "What are the highlights of this text?",
        "Provide an abstract of the content.",
        "Write a brief account of what the document says.",
        "Summarize this document into its main themes.",
        "Provide a synopsis of the article.",
        "What are the critical points made in this text?",
        "Condense the article into a short summary.",
        "What is the summary of this document?"
    ],
    "segmentation": [
        "Segment the regions of a pet in this image.",
        "Highlight the areas containing pets in the picture.",
        "Draw boundaries around the pets in the image.",
        "What are the segmented regions of this image?",
        "Segment this image into pet-related regions.",
        "Divide this image into regions for each pet.",
        "What parts of the image contain pets?",
        "Mark the pets in this image.",
        "Which areas of the image represent pets?",
        "Outline the pets visible in this picture.",
        "Segment the objects in the image and highlight the pets.",
        "Identify the pet regions in the image.",
        "Create a segmentation map of this image.",
        "Draw boundaries around all the pets in this picture.",
        "Which regions in the image belong to the pets?",
        "Separate the pet regions from the background in this image.",
        "Create a segmented output for this pet image.",
        "Mark each pet in this image with a boundary.",
        "Segment the pets visible in this photo.",
        "Highlight the segmented regions of this image.",
        "Create a boundary map for the pets in this image.",
        "What regions of the image correspond to pets?",
        "Outline the pet areas in the image.",
        "Generate a segmentation mask for the pets.",
        "What are the pet regions in this picture?"
    ],
    "sentiment": [
        "Is this sentence positive or negative?",
        "Determine the sentiment of this statement.",
        "What is the sentiment of this text?",
        "Classify the emotion conveyed in this sentence.",
        "What is the emotional tone of this statement?",
        "Identify whether the sentiment is positive or negative.",
        "Analyze the sentiment of this text.",
        "Is the sentiment of this sentence positive, negative, or neutral?",
        "Assess the emotional tone of the sentence.",
        "What feeling does this sentence convey?",
        "Categorize the sentiment of this statement.",
        "Classify this sentence as having positive or negative sentiment.",
        "What is the mood expressed in this sentence?",
        "Does this text express positive or negative sentiment?",
        "Determine if this text is optimistic or pessimistic.",
        "Evaluate the sentiment of this text snippet.",
        "What is the emotional content of this sentence?",
        "Analyze the tone of this statement.",
        "Is the sentiment in this sentence good or bad?",
        "Categorize the emotion in this sentence.",
        "Label this sentence as positive or negative.",
        "What is the sentiment score of this sentence?",
        "Identify the polarity of this statement.",
        "Determine the positive or negative sentiment of the text.",
        "What emotional response does this sentence evoke?"
    ],
    "classification": [
        "Classify the objects in the image into categories.",
        "Identify the type of object in this image.",
        "What class does this object belong to?",
        "Categorize the object shown in the image.",
        "Classify the image into one of several categories.",
        "What is the object type in this image?",
        "Which category does this image fall under?",
        "Classify this photo into its appropriate category.",
        "What is the class of the object in this picture?",
        "Label the object in the image with its category.",
        "Determine the category of this image.",
        "What type of item is displayed in this picture?",
        "Identify the classification for this image.",
        "What is the label for this image?",
        "Categorize this image into its appropriate class.",
        "Classify the contents of this image.",
        "Which category does this photo belong to?",
        "Determine the correct class for this image.",
        "What label should be assigned to this image?",
        "Identify the object in this photo by its type.",
        "Classify the objects in this picture.",
        "What type of object does this image show?",
        "Identify the object class in this image.",
        "Categorize the image based on its content.",
        "Label this image with the correct class."
    ]
}


These text files MUST be downloaded manually, as they are custom-made .txt files.

In [ ]:
with open('summarization.txt', 'r') as file:
    for line in file:
        TASK_PROMPTS["summarization"].append(line.strip())

with open('sentiment_analysis.txt', 'r') as file:
    for line in file:
        TASK_PROMPTS["sentiment"].append(line.strip())

with open('segmentation.txt', 'r') as file:
    for line in file:
        TASK_PROMPTS["segmentation"].append(line.strip())

with open('captioning.txt', 'r') as file:
    for line in file:
        TASK_PROMPTS["captioning"].append(line.strip())

with open('classification.txt', 'r') as file:
    for line in file:
        TASK_PROMPTS["classification"].append(line.strip())

In [ ]:
TASK_PROMPTS

{'captioning': ['Write a caption for this image.',
  'Describe what is happening in the picture.',
  'What is shown in the image?',
  'Explain the content of this image in a sentence.',
  'Provide a one-sentence description of this image.',
  'Describe the scene in the image.',
  'What does the image depict?',
  'Write a descriptive caption for the picture.',
  'Summarize the content of the image.',
  'Generate a caption for the given image.',
  'What activity is occurring in the picture?',
  'Provide a sentence describing the objects and scene in the image.',
  'What is the main focus of this image?',
  'Generate a natural language description of this image.',
  'Explain the image in your own words.',
  'What event is taking place in the picture?',
  'Describe the objects and actions in the image.',
  'Provide a short description of this photograph.',
  'Write a caption that explains the image.',
  'Summarize the activity or objects in the picture.',
  'What does the photograph show?'

In [ ]:
tasks = {
    "captioning": 0,
    "summarization": 1,
    "segmentation": 2,
    "sentiment": 3,
    "classification": 4
}

In [ ]:
split_prompts = {}
for task, prompts in TASK_PROMPTS.items():
    random.shuffle(prompts)  # Shuffle prompts for randomness
    train, temp = train_test_split(prompts, test_size=0.3, random_state=42)
    val, test = train_test_split(temp, test_size=0.5, random_state=42)
    split_prompts[task] = {"train": train, "val": val, "test": test}

In [ ]:
def create_split_dataset(split):
    texts = []
    labels = []
    for task, splits in split_prompts.items():
        texts.extend(splits[split])
        labels.extend([tasks[task]] * len(splits[split]))
    return texts, labels


In [ ]:
train_texts, train_labels = create_split_dataset("train")
val_texts, val_labels = create_split_dataset("val")
test_texts, test_labels = create_split_dataset("test")

# Verify split sizes
print(f"Training size: {len(train_texts)}, Validation size: {len(val_texts)}, Test size: {len(test_texts)}")


Training size: 435, Validation size: 95, Test size: 95


In [ ]:
from collections import Counter
print("Train distribution:", Counter(train_labels))
print("Validation distribution:", Counter(val_labels))
print("Test distribution:", Counter(test_labels))

Train distribution: Counter({0: 87, 1: 87, 2: 87, 3: 87, 4: 87})
Validation distribution: Counter({0: 19, 1: 19, 2: 19, 3: 19, 4: 19})
Test distribution: Counter({0: 19, 1: 19, 2: 19, 3: 19, 4: 19})


### 4.3 Task Classification Dataset

In [ ]:
class TaskClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        """
        :param texts: List of text inputs.
        :param labels: List of task labels (0-4).
        :param tokenizer: Pretrained tokenizer (e.g., BERT tokenizer).
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

        # Assign a padding token if it's not set
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token or '[PAD]'


    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=128,
            return_tensors="pt"
        )
        tokenized = {key: val.squeeze(0) for key, val in tokenized.items()}
        return tokenized, label


In [ ]:
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Prepare datasets
train_dataset = TaskClassificationDataset(train_texts, train_labels, tokenizer)
val_dataset = TaskClassificationDataset(val_texts, val_labels, tokenizer)
test_dataset = TaskClassificationDataset(test_texts, test_labels, tokenizer)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

### 4.4 Training Pipeline

In [ ]:
def train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=5, device="cuda"):
    """
    Train the Task Classifier model.
    :param model: The TaskClassifier model.
    :param train_loader: DataLoader for training data with weighted sampling.
    :param val_loader: DataLoader for validation data.
    :param optimizer: Optimizer for the model.
    :param criterion: Loss function (e.g., CrossEntropyLoss).
    :param num_epochs: Number of epochs to train.
    :param device: Device to run training on.
    """
    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for batch in train_loader:
            # Unpack batch
            tokenized_text, labels = batch
            tokenized_text = {key: val.to(device) for key, val in tokenized_text.items()}
            labels = labels.to(device)

            # Forward pass
            task_logits, _ = model(tokenized_text)  # Extract task_logits
            loss = criterion(task_logits, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Compute accuracy
            preds = torch.argmax(task_logits, dim=-1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

        train_accuracy = correct / total
        train_loss = total_loss / len(train_loader)

        # Validate the model
        val_loss, val_accuracy = validate_model(model, val_loader, criterion, device)

        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}\n")


In [ ]:
def validate_model(model, val_loader, criterion, device="cuda"):
    """
    Validate the Task Classifier model.
    :param model: The trained TaskClassifier model.
    :param val_loader: DataLoader for validation data.
    :param criterion: Loss function (e.g., CrossEntropyLoss).
    :param device: Device to run validation on.
    :return: Validation loss and accuracy.
    """
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in val_loader:
            # Unpack batch
            tokenized_text, labels = batch
            tokenized_text = {key: val.to(device) for key, val in tokenized_text.items()}
            labels = labels.to(device)

            # Forward pass
            task_logits, _ = model(tokenized_text)  # Extract task_logits
            loss = criterion(task_logits, labels)
            total_loss += loss.item()

            # Compute accuracy
            preds = torch.argmax(task_logits, dim=-1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(val_loader)
    accuracy = correct / total
    return avg_loss, accuracy


In [ ]:
def test_model(model, test_loader, device="cuda"):
    """
    Test the Task Classifier model.
    :param model: The trained TaskClassifier model.
    :param test_loader: DataLoader for test data.
    :param device: Device to run testing on.
    :return: Test accuracy.
    """
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in test_loader:
            # Unpack batch
            tokenized_text, labels = batch
            tokenized_text = {key: val.to(device) for key, val in tokenized_text.items()}
            labels = labels.to(device)

            # Forward pass
            task_logits, _ = model(tokenized_text)  # Extract task_logits
            preds = torch.argmax(task_logits, dim=-1)

            # Compute accuracy
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return accuracy


In [ ]:
# Initialize the model
task_classifier = TaskClassifier(num_tasks=5).to("cuda")

# Define optimizer and loss function
optimizer = Adam(task_classifier.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

# Train the model
train_model(task_classifier, train_loader, val_loader, optimizer, criterion, num_epochs=10, device="cuda")


Epoch 1/10
Train Loss: 1.5866, Train Accuracy: 0.2690
Validation Loss: 1.5107, Validation Accuracy: 0.4947

Epoch 2/10
Train Loss: 1.4214, Train Accuracy: 0.6253
Validation Loss: 1.2701, Validation Accuracy: 0.7895

Epoch 3/10
Train Loss: 1.1529, Train Accuracy: 0.8069
Validation Loss: 0.9628, Validation Accuracy: 0.9263

Epoch 4/10
Train Loss: 0.8498, Train Accuracy: 0.9494
Validation Loss: 0.6808, Validation Accuracy: 0.9895

Epoch 5/10
Train Loss: 0.5842, Train Accuracy: 0.9770
Validation Loss: 0.4452, Validation Accuracy: 0.9789

Epoch 6/10
Train Loss: 0.3767, Train Accuracy: 0.9954
Validation Loss: 0.2987, Validation Accuracy: 0.9895

Epoch 7/10
Train Loss: 0.2593, Train Accuracy: 0.9954
Validation Loss: 0.1923, Validation Accuracy: 1.0000

Epoch 8/10
Train Loss: 0.1761, Train Accuracy: 0.9977
Validation Loss: 0.1473, Validation Accuracy: 1.0000

Epoch 9/10
Train Loss: 0.1250, Train Accuracy: 0.9977
Validation Loss: 0.1222, Validation Accuracy: 0.9895

Epoch 10/10
Train Loss: 0.09

In [ ]:
# Test the model
test_accuracy = test_model(task_classifier, test_loader, device="cuda")
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.9895


In [ ]:
torch.save(task_classifier.state_dict(), "task_classifier.pth")

# 5. Define the Mixture-of-Experts (MoE) Module

The MoE module contains several task-specific experts, each designed for a specific task, and a generalist to handle low confidence classification. Our `TaskClassifer` above activates one expert based on the task.

### 5.1 Evaluation of Fine-Tuned Text Task Expert Modules

1. Fine-Tuned Bart for Summarization

Comparing Fine-Tuned model with vanilla model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import transformers
import datasets

In [ ]:
import pandas as pd
from datasets import DatasetDict, load_dataset

# Convert DataFrames to Hugging Face Datasets
dataset_train = load_dataset("samsum", split="train")
dataset_test = load_dataset("samsum", split="test")
dataset_val = load_dataset("samsum", split="validation")

# Create DatasetDict
dataset_dict = DatasetDict({
    'train': dataset_train,
    'test': dataset_test,
    'validation': dataset_val
})

print(dataset_dict)
dataset_samsum = dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})


In [ ]:
dataset_samsum["train"][:10]

{'id': ['13818513',
  '13728867',
  '13681000',
  '13730747',
  '13728094',
  '13716343',
  '13611672',
  '13730463',
  '13809976',
  '13809912'],
 'dialogue': ["Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
  'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great',
  "Tim: Hi, what's up?\r\nKim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating\r\nTim: What did you plan on doing?\r\nKim: Oh you know, uni stuff and unfucking my room\r\nKim: Maybe tomorrow I'll move my ass and do everything\r\nKim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies\r\nTim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores\r\nTim: It really helps\r\nKim: thanks, maybe I'll do that\r\nTim: I also like using post-its in kaban style",
  "Edward: Rachel, I think I'm in ove with Bella..\r\nrachel: 

In [ ]:
# Get lengths of train, test and val data
split_train_test_val = [len(dataset_samsum[split]) for split in dataset_samsum]

In [ ]:
split_train_test_val

[14732, 819, 818]

In [ ]:
print(f"Split lengths: {split_train_test_val}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")
print(dataset_samsum["test"][0]["dialogue"])
print("\nSummary:")
print(dataset_samsum["test"][0]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model_ckpt = "dhivyeshrk/bart-large-cnn-samsum"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
!pip install rouge_score evaluate

In [ ]:
import evaluate
from tqdm import tqdm

rouge_metric = evaluate.load("rouge")
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

In [ ]:
def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [ ]:
def evaluate_summaries(dataset, metric, model, tokenizer, batch_size=16, device=device,
                                   column_text="dialogue",
                                   column_summary="summary"):
    '''Calculate respective rouge metric for the given data'''
    article_batches = list(chunks(dataset[column_text], batch_size)) # dialogue batches
    target_batches = list(chunks(dataset[column_summary], batch_size))  # target batches

    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                            padding="max_length", return_tensors="pt") # encode the input
            print(type(article_batch))
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),  # generate summary
                             attention_mask=inputs["attention_mask"].to(device),
                             length_penalty=0.8, num_beams=8, max_length=128)
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True) for s in summaries] # decode them

            decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]  # misc processing
            metric.add_batch(predictions=decoded_summaries, references=target_batch)  # add this batch to the metric

    score = metric.compute() # Calculate final metric score

    return score

In [ ]:
score_fineTuned = evaluate_summaries(
    dataset_samsum["test"],
    rouge_metric,
    model,
    tokenizer,
    column_text="dialogue",
    column_summary="summary",
    batch_size=8
)


  0%|          | 0/103 [00:00<?, ?it/s]

<class 'list'>


  1%|          | 1/103 [00:03<06:17,  3.70s/it]

<class 'list'>


  2%|▏         | 2/103 [00:06<05:20,  3.17s/it]

...

 98%|█████████▊| 101/103 [05:07<00:06,  3.07s/it]

<class 'list'>


 99%|█████████▉| 102/103 [05:10<00:03,  3.05s/it]

<class 'list'>


100%|██████████| 103/103 [05:11<00:00,  3.03s/it]


In [ ]:
rouge_dict_fineTuned = dict((rn, score_fineTuned[rn]) for rn in rouge_names)


In [ ]:
pd.DataFrame.from_records(rouge_dict_fineTuned, index=[f"bart-cnn-FineTuned"])


,rouge1,rouge2,rougeL,rougeLsum
bart-cnn-FineTuned,0.392169,0.197575,0.302004,0.301908


In [ ]:
torch.cuda.empty_cache()  # Free GPU Memory to test on original model.

In [ ]:
model_ckpt = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
score_original = evaluate_summaries(
    dataset_samsum["test"],
    rouge_metric,
    model,
    tokenizer,
    column_text="dialogue",
    column_summary="summary",
    batch_size=8
)
rouge_dict_original = dict((rn, score_original[rn]) for rn in rouge_names)


  0%|          | 0/103 [00:00<?, ?it/s]

<class 'list'>


  1%|          | 1/103 [00:03<05:58,  3.52s/it]

<class 'list'>


  2%|▏         | 2/103 [00:06<05:44,  3.41s/it]

...

 98%|█████████▊| 101/103 [06:13<00:07,  3.63s/it]

<class 'list'>


 99%|█████████▉| 102/103 [06:16<00:03,  3.67s/it]

<class 'list'>


100%|██████████| 103/103 [06:18<00:00,  3.68s/it]


In [ ]:
pd.DataFrame.from_records(rouge_dict_original, index=[f"bart-cnn-Facebook"])

,rouge1,rouge2,rougeL,rougeLsum
bart-cnn-Facebook,0.300203,0.097865,0.226568,0.226614


In [ ]:
pd.DataFrame.from_records(rouge_dict_fineTuned, index=[f"bart-cnn-FineTuned"])

,rouge1,rouge2,rougeL,rougeLsum
bart-cnn-FineTuned,0.392169,0.197575,0.302004,0.301908


We can see here that we see a significant increase in all Rouge metrics. The model successfully demonstrates specialization in summarization tasks.

2. Fine-Tuned BERT for Sentiment Analysis

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
import evaluate

In [ ]:
import wandb
wandb.init(project="bert-base-cased-fine-tuned-sst2", name="ckandrew04")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ckandrew04 (cs4650). Use `wandb login --relogin` to force relogin


In [ ]:
dataset = load_dataset("glue", "sst2")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding=True, max_length=128)


In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset = encoded_dataset.remove_columns(["sentence", "idx"])
encoded_dataset.set_format(type="torch")

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="bert-base-cased-fine-tuned-sst2",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="wandb"
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-59-10d880e057ed>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.180900,0.242000,0.915138
2,0.127500,0.340481,0.910550
3,0.073600,0.365363,0.917431


TrainOutput(global_step=12630, training_loss=0.14392916890051471, metrics={'train_runtime': 799.7679, 'train_samples_per_second': 252.632, 'train_steps_per_second': 15.792, 'total_flos': 7246198200700140.0, 'train_loss': 0.14392916890051471, 'epoch': 3.0})

In [ ]:
# trainer.push_to_hub("Training done for bert-base-cased-fine-tuned-sst2")

In [ ]:
results = trainer.evaluate(encoded_dataset["validation"])
print(f"Fine-Tuned BERT-base-cased Accuracy: {results['eval_accuracy']:.4f}")

Fine-Tuned BERT-base-cased Accuracy: 0.9174


In [ ]:
val_dataset = dataset["validation"]

fine_tuned_model_name = "ckandrew04/bert-base-cased-fine-tuned-sst2"
base_model_name = "bert-base-cased"

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_name)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [ ]:
def preprocess_data(dataset):
    tokenized = tokenizer(
        dataset["sentence"],
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    labels = torch.tensor(dataset["label"])
    return tokenized, labels

In [ ]:
val_inputs, val_labels = preprocess_data(val_dataset)

In [ ]:
batch_size = 16
val_loader = DataLoader(list(zip(val_inputs["input_ids"], val_inputs["attention_mask"], val_labels)), batch_size=batch_size)

In [ ]:
def evaluate_model(model, data_loader, device="cpu"):
    model.to(device)
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
print("Evaluating Fine-Tuned Model...")
fine_tuned_val_accuracy = evaluate_model(fine_tuned_model, val_loader, device)

# Print results
print(f"Fine-Tuned BERT-base-cased Accuracy: {fine_tuned_val_accuracy:.4f}")

Evaluating Fine-Tuned Model...
Fine-Tuned BERT-base-cased Accuracy: 0.9209


We have successfully fine-tuned the bert-base-cased model using the SST2 dataset, for Sentiment Analysis. We achieved accuracy of 0.9209. The model can be found under Minjun's Hugging Face profile, `ckandrew04/bert-base-cased-fine-tuned-sst2`.

### 5.2 Mixture of Experts Module definition

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForImageClassification, AutoModelForSemanticSegmentation, VisionEncoderDecoderModel


In [ ]:
class MixtureOfExperts(nn.Module):
    def __init__(self, task_classifier, experts, generalist, device="cuda"):
        super(MixtureOfExperts, self).__init__()
        self.task_classifier = task_classifier.to(device)
        self.experts = experts
        self.generalist = generalist.to(device)
        self.device = device

        # Mapping of integer labels to task names
        self.label_to_task = {
            0: "Image Captioning",
            1: "Summarization",
            2: "Image Segmentation",
            3: "Sentiment Analysis",
            4: "Image Classification"
        }

    def forward(self, inputs, task_type=None):
        """
        Forward pass for the Mixture of Experts.
        :param inputs: Input data for the task (text or image or both).
        :param task_type: Optional manual override for the task type.
        :return: Model outputs based on the routed expert or generalist.
        """
        print(f"Inputs received: {inputs}")
        print(f"Type of inputs: {type(inputs)}")

        # If task_type is manually provided
        if task_type is not None:
            if task_type in self.experts:
                return self._route_to_expert(task_type, inputs)
            else:
                return self.generalist(inputs)

        # Use TaskClassifier to determine task type
        if "prompt" in inputs:  # Assuming text prompt for task classification
            tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
            tokenized_prompt = tokenizer(
                inputs["prompt"],
                padding="max_length",
                truncation=True,
                max_length=128,
                return_tensors="pt"
            )
            tokenized_prompt = {key: val.to(self.device) for key, val in tokenized_prompt.items()}
            task_logits, confidences = self.task_classifier(tokenized_prompt)
            task_idx = torch.argmax(confidences, dim=-1).item()
            task_type = self.label_to_task[task_idx]
            return self._route_to_expert(task_type, inputs["input"])  # Pass only the actual input
        else:
            raise ValueError("Inputs must contain a 'prompt' for task classification.")

    def _route_to_expert(self, task_type, actual_input):
        """
        Route the inputs to the appropriate expert or generalist and process the output.
        :param task_type: Type of the task.
        :param actual_input: Actual input data for the task.
        :return: Processed expert or generalist output.
        """
        print(f"Routing to expert for task: {task_type}")

        # If task type corresponds to an expert, route to the expert
        expert = self.experts.get(task_type)
        if expert:
            # Process inputs based on expert type (text or image)
            if task_type == "Summarization":
                tokenizer = AutoTokenizer.from_pretrained("dhivyeshrk/bart-large-cnn-samsum")
                if isinstance(actual_input, str):
                    tokenized_input = tokenizer(
                        actual_input,
                        padding="longest",  # Use longest padding
                        truncation=True,
                        max_length=512,
                        return_tensors="pt"
                    )
                    tokenized_input.pop("token_type_ids", None)

                    tokenized_input["attention_mask"] = tokenized_input["input_ids"].ne(tokenizer.pad_token_id)
                    tokenized_input = {key: val.to(self.device) for key, val in tokenized_input.items()}

                    outputs = expert.generate(
                        **tokenized_input,
                        max_new_tokens=100,  # Set maximum output length for generation
                        no_repeat_ngram_size=2,  # Avoid repetition
                        early_stopping=True
                    )
                    return tokenizer.decode(outputs[0], skip_special_tokens=True)
                else:
                    raise ValueError("For Summarization, `actual_input` must be a string.")

            elif task_type == "Sentiment Analysis":
                tokenizer = AutoTokenizer.from_pretrained("ckandrew04/bert-base-cased-fine-tuned-sst2")
                if isinstance(actual_input, str):
                    tokenized_input = tokenizer(
                        actual_input,
                        padding="max_length",
                        truncation=True,
                        max_length=128,
                        return_tensors="pt"
                    )
                    tokenized_input = {key: val.to(self.device) for key, val in tokenized_input.items()}
                    outputs = expert(**tokenized_input)
                    predicted_label = torch.argmax(outputs.logits, dim=-1).item()

                    print(outputs)

                    print(predicted_label)

                    return "Positive" if predicted_label == 1 else "Negative"
                else:
                    raise ValueError("For Sentiment Analysis, `actual_input` must be a string.")

            elif task_type == "Image Classification":
                if isinstance(actual_input, torch.Tensor):
                    actual_input = actual_input.to(self.device)
                    outputs = expert(pixel_values=actual_input)
                    predicted_label = torch.argmax(outputs.logits, dim=-1).item()
                    return expert.config.id2label[predicted_label]  # Map index to class label
                else:
                    raise ValueError("For Image Classification, `actual_input` must be a torch.Tensor.")

            elif task_type == "Image Segmentation":
                if isinstance(actual_input, torch.Tensor):
                    actual_input = actual_input.to(self.device)
                    outputs = expert(pixel_values=actual_input)
                    return outputs.logits.argmax(dim=1)  # Return the segmentation mask
                else:
                    raise ValueError("For Image Segmentation, `actual_input` must be a torch.Tensor.")

            elif task_type == "Image Captioning":
                if isinstance(actual_input, torch.Tensor):
                    actual_input = actual_input.to(self.device)
                    outputs = expert.generate(pixel_values=actual_input)
                    tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
                    return tokenizer.decode(outputs[0], skip_special_tokens=True)
                else:
                    raise ValueError("For Image Captioning, `actual_input` must be a torch.Tensor.")

        # Fallback to generalist if task type does not match any expert
        elif self.generalist:
            tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
            if isinstance(actual_input, str):
                tokenized_input = tokenizer(
                    actual_input,
                    padding="max_length",
                    truncation=True,
                    max_length=128,
                    return_tensors="pt"
                )
                tokenized_input = {key: val.to(self.device) for key, val in tokenized_input.items()}
                outputs = self.generalist(**tokenized_input)
                mask_token_index = (tokenized_input["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)
                predicted_tokens = torch.argmax(outputs.logits, dim=-1)
                return tokenizer.decode(predicted_tokens[mask_token_index], skip_special_tokens=True)
            else:
                raise ValueError("For generalist tasks, `actual_input` must be a string.")

        else:
            raise ValueError(f"No expert or generalist found for task type: {task_type}")


In [ ]:
from transformers import (
    BertTokenizer, BertForSequenceClassification, BartForConditionalGeneration,
    ViTForImageClassification, SegformerForSemanticSegmentation, VisionEncoderDecoderModel,
    AutoModelForMaskedLM
)
import torch.nn as nn

# Task Classifier
task_classifier = TaskClassifier(num_tasks=5)
# Load pre-trained or trained weights
task_classifier.load_state_dict(torch.load("task_classifier.pth"))  # Replace with your saved weights path
task_classifier.eval()  # Ensure the model is in evaluation mode

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Experts for each task
experts = {
    "Summarization": AutoModelForSeq2SeqLM.from_pretrained("dhivyeshrk/bart-large-cnn-samsum").to(device),  # Fine-tuned BART
    "Sentiment Analysis": AutoModelForSequenceClassification.from_pretrained("ckandrew04/bert-base-cased-fine-tuned-sst2").to(device),  # Fine-tuned BERT
    "Image Classification": ViTForImageClassification.from_pretrained("google/vit-base-patch16-224").to(device),  # Vision Transformer
    "Image Segmentation": SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512").to(device),  # SegFormer model
    "Image Captioning": VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning").to(device)  # ViT + GPT-2 for captioning
}

# Generalist fallback model
generalist = AutoModelForMaskedLM.from_pretrained("bert-base-uncased").to(device)
# Generalist pre-trained multi-modal model, this must be replaced by a multi-modal generalist model, such as GPT-4o.
# Currently implemented as bert-base-cased for simplification. bert-base-cased is NOT a multimodal model.

# Mixture of Experts instantiation
moe = MixtureOfExperts(
    task_classifier=task_classifier,
    experts=experts,
    generalist=generalist,
    device=device
)


<ipython-input-88-f1d8cbc11a83>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  task_classifier.load_state_dict(torch.load("task_classifier.pth"))  # Replace with your sav

In [ ]:
sample_inputs = {
    "Summarization": {
        "prompt": "Provide a brief summary of the dialogue.",
        "input": "Hannah: Hey, do you have Betty's number? Amanda: Lemme check Hannah: <file_gif> Amanda: Sorry, can't find it. Amanda: Ask Larry Amanda: He called her last time we were at the park together Hannah: I don't know him well Hannah: <file_gif> Amanda: Don't be shy, he's very nice Hannah: If you say so.. Hannah: I'd rather you texted him Amanda: Just text him 🙂 Hannah: Urgh.. Alright Hannah: Bye Amanda: Bye bye"
    },
    "Sentiment Analysis": {
        "prompt": "Analyze the sentiment of the following review:",
        "input": "it takes a strange kind of laziness to waste the talents of robert forster , anne meara , eugene levy , and reginald veljohnson all in the same movie ."
    },
    "Image Classification": {
        "prompt": "Classify the following image:",
        "input": torch.rand(1, 3, 224, 224)  # Random image tensor simulating an RGB image
    },
    "Image Segmentation": {
        "prompt": "Segment the objects in the following image:",
        "input": torch.rand(1, 3, 224, 224)  # Random image tensor simulating an RGB image
    },
    "Image Captioning": {
        "prompt": "Generate a caption for the following image:",
        "input": torch.rand(1, 3, 224, 224)  # Random image tensor simulating an RGB image
    },
}

In [ ]:
# Testing with updated logic
for task_name, data in sample_inputs.items():
    print(f"\nTesting for Task: {task_name}")

    try:
        prompt = data["prompt"]
        actual_input = data["input"]

        # Prepare the inputs
        inputs = {"prompt": prompt, "input": actual_input}

        # Forward pass through MixtureOfExperts
        output = moe(inputs)
        print(f"Output for {task_name}: {output}")

    except Exception as e:
        print(f"Error during {task_name} testing: {e}")



Testing for Task: Summarization
Inputs received: {'prompt': 'Provide a brief summary of the dialogue.', 'input': "Hannah: Hey, do you have Betty's number? Amanda: Lemme check Hannah: <file_gif> Amanda: Sorry, can't find it. Amanda: Ask Larry Amanda: He called her last time we were at the park together Hannah: I don't know him well Hannah: <file_gif> Amanda: Don't be shy, he's very nice Hannah: If you say so.. Hannah: I'd rather you texted him Amanda: Just text him 🙂 Hannah: Urgh.. Alright Hannah: Bye Amanda: Bye bye"}
Type of inputs: <class 'dict'>
Routing to expert for task: Summarization
Output for Summarization: Hannah is looking for Betty's number. Amanda suggests Hannah to ask Larry if he called Betty last time they were at the park together. Hannah doesn't know him well, but he's very nice. She should text him.   Hannah and Amanda agree to do that.

Testing for Task: Sentiment Analysis
Inputs received: {'prompt': 'Analyze the sentiment of the following review:', 'input': 'it tak

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Output for Image Captioning: a blurry photo of a single bird flying through the air 


# 6. Combine Components

The final model integrates the MoM backbone, Task Classifier, and MoE module. The MoM separate inputs from different modalities, using only what is needed, the Task Classifier predicts the task, and the MoE generates task-specific outputs.

In [ ]:
import torch
import torch.nn as nn

class MoMMoE(nn.Module):
    def __init__(self, mom, moe):
        super(MoMMoE, self).__init__()
        self.mom = mom  # Mixture of Modalities
        self.moe = moe  # Mixture of Experts

    def forward(self, raw_inputs, task_type=None):
        """
        Forward pass for the integrated model.
        :param raw_inputs: Raw dictionary of text or image input.
        :param task_type: Optional manual override for task type.
        :return: Model output from the Mixture of Experts.
        """
        # Process inputs through MoM
        processed_inputs = self.mom(raw_inputs)

        # Pass processed inputs to MoE
        output = self.moe(processed_inputs, task_type=task_type)
        return output

In [ ]:
sample_inputs = {
    "Summarization": {
        "text": "Provide a brief summary of the dialogue. Hannah: Hey, do you have Betty's number? Amanda: Lemme check Hannah: <file_gif> Amanda: Sorry, can't find it. Amanda: Ask Larry Amanda: He called her last time we were at the park together Hannah: I don't know him well Hannah: <file_gif> Amanda: Don't be shy, he's very nice Hannah: If you say so.. Hannah: I'd rather you texted him Amanda: Just text him 🙂 Hannah: Urgh.. Alright Hannah: Bye Amanda: Bye bye"
    },
    "Sentiment Analysis": {
        "text": "Analyze the sentiment of the following review. it takes a strange kind of laziness to waste the talents of robert forster , anne meara , eugene levy , and reginald veljohnson all in the same movie ."
    },
    "Image Classification": {
        "text": "Classify the following image.",
        "image": torch.rand(1, 3, 224, 224)  # Random image tensor simulating an RGB image
    },
    "Image Segmentation": {
        "text": "Segment the objects in the following image.",
        "image": torch.rand(1, 3, 224, 224)  # Random image tensor simulating an RGB image
    },
    "Image Captioning": {
        "text": "Generate a caption for the following image.",
        "image": torch.rand(1, 3, 224, 224)  # Random image tensor simulating an RGB image
    }
}


In [ ]:
# Initialize MoM and MoE
mom = MixtureOfModalities()
moe = MixtureOfExperts(
    task_classifier=task_classifier,
    experts=experts,
    generalist=generalist,
    device=device
)

# Initialize FinalModel
mommoe = MoMMoE(mom, moe)



In [ ]:
# Testing with sample inputs
for task_name, raw_input in sample_inputs.items():
    print(f"\nTesting for Task: {task_name}")
    try:
        output = mommoe(raw_inputs=raw_input)
        print(f"Output for {task_name}: {output}")
    except Exception as e:
        print(f"Error during {task_name} testing: {e}")



Testing for Task: Summarization
Inputs received: {'prompt': 'Provide a brief summary of the dialogue.', 'input': "Hannah: Hey, do you have Betty's number? Amanda: Lemme check Hannah: <file_gif> Amanda: Sorry, can't find it. Amanda: Ask Larry Amanda: He called her last time we were at the park together Hannah: I don't know him well Hannah: <file_gif> Amanda: Don't be shy, he's very nice Hannah: If you say so.. Hannah: I'd rather you texted him Amanda: Just text him 🙂 Hannah: Urgh.. Alright Hannah: Bye Amanda: Bye bye"}
Type of inputs: <class 'dict'>
Routing to expert for task: Summarization
Output for Summarization: Hannah is looking for Betty's number. Amanda suggests Hannah to ask Larry if he called Betty last time they were at the park together. Hannah doesn't know him well, but he's very nice. She should text him.   Hannah and Amanda agree to do that.

Testing for Task: Sentiment Analysis
Inputs received: {'prompt': 'Analyze the sentiment of the following review.', 'input': 'it tak